In [ ]:
from google.colab import drive

drive.mount("/content/drive/")
!unzip /content/drive/MyDrive/Senior_project/klsg_dataset.zip

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
import glob
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

import torch
from torch import nn
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR

from torchvision import models
from torchvision.models import VGG19_Weights
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

device = "cuda" if torch.cuda.is_available() else "cpu"

## Setup

In [ ]:
df = None
for label in os.listdir("/content/klsg_dataset"):
    image_files = glob.glob(f"/content/klsg_dataset/{label}/*.jpg") + glob.glob(f"/content/klsg_dataset/{label}/*.png")
    temp = pd.DataFrame({"image_path": image_files})
    temp["label"] = label
    df = pd.concat([df, temp]) if df is not None else temp

print(df.shape)
df.head()

(1912, 2)


,image_path,label
0,/content/klsg_dataset/ship/ship-408.png,ship
1,/content/klsg_dataset/ship/ship-262.png,ship
2,/content/klsg_dataset/ship/ship-155.png,ship
3,/content/klsg_dataset/ship/ship-079.png,ship
4,/content/klsg_dataset/ship/ship-241.png,ship


In [ ]:
df['label'].value_counts()

,count
label,
mine,600
seafloor,578
ship,487
victim,181
plane,66


In [ ]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

train_df, val_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
train_df.reset_index(drop=True, inplace=True); val_df.reset_index(drop=True, inplace=True)

print(train_df.shape, val_df.shape)
train_df.head()

(1338, 2) (574, 2)


,image_path,label
0,/content/klsg_dataset/mine/mine-580.jpg,0
1,/content/klsg_dataset/ship/ship-192.png,3
2,/content/klsg_dataset/victim/victim-090.png,4
3,/content/klsg_dataset/ship/ship-131.png,3
4,/content/klsg_dataset/mine/mine-424.jpg,0


## Data Prep

In [ ]:
imagenet_stats = VGG19_Weights.IMAGENET1K_V1.transforms()

transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_stats.mean, std=imagenet_stats.std),
])

class SeabedDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        image = Image.open(img_path).convert('RGB')
        labels = torch.tensor(self.dataframe.loc[idx, 'label'])

        if self.transform:
            image = self.transform(image)

        return image, labels

In [ ]:
# Create DataLoaders
train_dataset = SeabedDataset(train_df, transform=transform)
val_dataset = SeabedDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

for batch, label in train_loader:
    print(batch.shape)
    print(label.shape)
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


## Model Training

In [ ]:
class VGG19(nn.Module):
    def __init__(self):
        super().__init__()

        # load and enable pre-trained parameters for training
        self.backbone = models.vgg19(weights="DEFAULT")
        for params in self.backbone.parameters():
            params.requires_grad = True

        self.fc1 = nn.Linear(1000, 64)  # output_features from pretrained=1000
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, len(df['label'].unique())) # Use len(label_map) for the number of output classes

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, 1) # Flatten the output of the backbone
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

# Initialize model
model = VGG19() # Instantiate the class
model = model.to(device) # Move the instance to the device
model = nn.DataParallel(model)

criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=1e-4, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# summary(model, train_dataset[0][0].shape)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


100%|██████████| 548M/548M [00:12<00:00, 45.1MB/s]


In [ ]:
total_acc_train, total_loss_train = [], []
total_acc_val, total_loss_val = [], []

epochs = 10
for epoch in tqdm(range(epochs)):
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to the correct device
        logit = model(inputs)

        batch_correct = (torch.argmax(logit, axis=1) == labels).sum().item()
        loss = criterion(logit, labels)

        train_acc += batch_correct
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device) # Move inputs and labels to the correct device
            logit = model(inputs).to(device)

            batch_correct = (torch.argmax(logit, axis=1) == labels).sum().item()
            loss = criterion(logit, labels)

            val_acc += batch_correct
            val_loss += loss.item()

    # finalize metrics
    train_acc = round(train_acc/train_dataset.__len__(), 4)
    train_loss = round(train_loss/1000, 4)
    val_acc = round(val_acc/val_dataset.__len__(), 4)
    val_loss = round(val_loss/1000, 4)

    # store score of epoch
    total_acc_train.append(train_acc)
    total_acc_val.append(val_acc)
    total_loss_train.append(train_loss)
    total_loss_val.append(val_loss)

    print(f"\n=========== Epoch {epoch+1} ===========")
    print(f"Train: acc -> {train_acc} / loss -> {train_loss}")
    print(f"Val: acc -> {val_acc} / loss -> {val_loss}")

 10%|█         | 1/10 [00:28<04:16, 28.46s/it]


=========== Epoch 1 ===========
Train: acc -> 0.4417 / loss -> 0.0569
Val: acc -> 0.6707 / loss -> 0.017


 20%|██        | 2/10 [00:57<03:48, 28.55s/it]


=========== Epoch 2 ===========
Train: acc -> 0.7601 / loss -> 0.03
Val: acc -> 0.7944 / loss -> 0.0117


 30%|███       | 3/10 [01:25<03:18, 28.30s/it]

...


=========== Epoch 8 ===========
Train: acc -> 0.9155 / loss -> 0.0119
Val: acc -> 0.9007 / loss -> 0.0053


 90%|█████████ | 9/10 [04:13<00:28, 28.17s/it]


=========== Epoch 9 ===========
Train: acc -> 0.9223 / loss -> 0.011
Val: acc -> 0.9181 / loss -> 0.0047


100%|██████████| 10/10 [04:41<00:00, 28.20s/it]


=========== Epoch 10 ===========
Train: acc -> 0.9215 / loss -> 0.0109
Val: acc -> 0.9181 / loss -> 0.0048
